In [1]:
import pandas as pd
from pandasql import sqldf
import numpy as np

### Problem: 1731. The Number of Employees Who Report to Each Employee

**Table**: `Employees`

| Column Name  | Type    |
|--------------|---------|
| employee_id  | int     |
| name         | varchar |
| reports_to   | int     |
| age          | int     |

- `employee_id` is the column with unique values for this table.
- This table contains information about the employees and the ID of the manager they report to.
- Some employees do not report to anyone (`reports_to` is null).

For this problem, we consider a **manager** as an employee who has at least 1 other employee reporting to them.

### Task:
Write a solution to report the IDs and the names of all managers, the number of employees who report directly to them, and the average age of the reports rounded to the nearest integer.

Return the result table ordered by `employee_id`.

### Example:

**Input**: 
`Employees` table:

| employee_id | name    | reports_to | age |
|-------------|---------|------------|-----|
| 1           | Michael | null       | 45  |
| 2           | Alice   | 1          | 38  |
| 3           | Bob     | 1          | 42  |
| 4           | Charlie | 2          | 34  |
| 5           | David   | 2          | 40  |
| 6           | Eve     | 3          | 37  |
| 7           | Frank   | null       | 50  |
| 8           | Grace   | null       | 48  |

**Output**:

| employee_id | name    | reports_count | average_age |
|-------------|---------|---------------|-------------|
| 1           | Michael | 2             | 40          |
| 2           | Alice   | 2             | 37          |
| 3           | Bob     | 1             | 37          |

The output lists the `employee_id` and `name` of each manager, along with `reports_count` (the number of direct reports) and `average_age` (the average age of those who report to them, rounded to the nearest integer). The result is sorted by `employee_id`.

In [ ]:
SELECT e1.employee_id, 
       e1.NAME, 
       Count(e2.reports_to)  AS reports_count, 
       Round(Avg(e2.age), 0) AS average_age 
FROM   employees e1 
       INNER JOIN employees e2 
               ON e1.employee_id = e2.reports_to 
GROUP  BY e1.employee_id, 
          e1.NAME 
ORDER  BY e1.employee_id ASC; 

In [ ]:
df = pd.read_csv('../data/1731_employees.csv') # loads the example

In [ ]:
#aux = df.dropna(subset=['reports_to'])

aux = df.copy()
#aux.loc[:,'average_age'] = aux.groupby('reports_to')['age'].transform('mean')

# Group employees by their manager to calculate the count of reports and the 
# average age
managers = aux.groupby('reports_to').agg(
    average_age = ('age', 'mean'),
    reports_count = ('name','count')
    ).reset_index()
managers.rename( columns = {'reports_to': 'employee_id' }, inplace = True)

# Adjust for banker's rounding by adding a very small number before rounding
managers[['average_age']] = (managers[['average_age']] + 1e-12).round()

# Merge the aggregated data with the original employees DataFrame to get the 
# names of managers
managers.merge(
    df,
    on = 'employee_id',
    how = 'left'
)[['employee_id', 'name', 'reports_count', 'average_age']]




### Problem: 176. Second Highest Salary

**Table**: `Employee`

| Column Name | Type |
|-------------|------|
| id          | int  |
| salary      | int  |

- `id` is the **primary key** (column with unique values) for this table.
- Each row of this table contains information about the **salary** of an employee.

### Task:
Write a solution to find the **second highest distinct salary** from the `Employee` table. If there is **no second highest salary**, return **null** (or **None** in Pandas).

### Example:

**Example 1:**

**Input**: 
`Employee` table:

| id | salary |
|----|--------|
| 1  | 100    |
| 2  | 200    |
| 3  | 300    |

**Output**:

| SecondHighestSalary |
|---------------------|
| 200                 |

**Example 2:**

**Input**: 
`Employee` table:

| id | salary |
|----|--------|
| 1  | 100    |

**Output**:

| SecondHighestSalary |
|---------------------|
| null                |

The output should list the **second highest distinct salary**. If no such salary exists, the result should be **null**.

In [ ]:
Two Solutions:

SELECT 
    IFNULL((
        SELECT DISTINCT salary 
        FROM Employee
        ORDER BY salary DESC
        LIMIT 1 OFFSET 1
    ), NULL) AS SecondHighestSalary;

SELECT
    max(salary) as SecondHighestSalary,
FROM employee
WHERE salary < (
    SELECT max(salary) FROM employee
);

In [ ]:
employee = pd.DataFrame({
    'id': [1,2],
    'salary': [100, 100]
})

employee

In [ ]:
# First, get only distinct salaries

aux = employee.drop_duplicates(['salary'])
aux = aux.sort_values('salary', ascending = False).reset_index()

if aux.shape[0] >= 2:
    aux['SecondHighestSalary'] = aux.loc[1,'salary']
    aux = aux[['SecondHighestSalary']].drop_duplicates()
else:
    aux['SecondHighestSalary'] = None

aux[['SecondHighestSalary']]



### Problem: 1193. Monthly Transactions I

**Table**: `Transactions`

| Column Name | Type    |
|-------------|---------|
| id          | int     |
| country     | varchar |
| state       | enum    |
| amount      | int     |
| trans_date  | date    |

- `id` is the **primary key** of this table.
- The table contains information about **incoming transactions**.
- The `state` column is an **enum** of type `["approved", "declined"]`.

### Task:
Write an SQL query to find, for each **month** and **country**, the **number of transactions** and their **total amount**, the **number of approved transactions** and their **total amount**.

Return the result table in any order.

### Example:

**Input**: 
`Transactions` table:

| id  | country | state    | amount | trans_date |
|-----|---------|----------|--------|------------|
| 121 | US      | approved | 1000   | 2018-12-18 |
| 122 | US      | declined | 2000   | 2018-12-19 |
| 123 | US      | approved | 2000   | 2019-01-01 |
| 124 | DE      | approved | 2000   | 2019-01-07 |

**Output**:

| month   | country | trans_count | approved_count | trans_total_amount | approved_total_amount |
|---------|---------|-------------|----------------|--------------------|-----------------------|
| 2018-12 | US      | 2           | 1              | 3000               | 1000                  |
| 2019-01 | US      | 1           | 1              | 2000               | 2000                  |
| 2019-01 | DE      | 1           | 1              | 2000               | 2000                  |

The output should include, for each **month** (in `YYYY-MM` format) and **country**:
- `trans_count`: The **total number of transactions**.
- `approved_count`: The **number of approved transactions**.
- `trans_total_amount`: The **sum of amounts** for all transactions.
- `approved_total_amount`: The **sum of amounts** for approved transactions.

In [ ]:
SELECT 
    DATE_FORMAT(trans_date, '%Y-%m') AS month
    country,
    COUNT(*) AS trans_count,
    SUM(amount) AS trans_total_amount,
    SUM( state = "approved") AS approved_count,
    SUM( (state = "approved")*amount)   AS approved_total_amount
FROM 
    Transactions
GROUP BY 
    month, country

--------------------------------------------------------------------------------

SELECT 
    DATE_FORMAT(trans_date, '%Y-%m') AS month,
    country, 
    COUNT(*) AS trans_count,
    SUM(amount) AS trans_total_amount,
    SUM(CASE WHEN state = 'approved' THEN 1 ELSE 0 END) AS approved_count,
    SUM(CASE WHEN state = 'approved' THEN amount ELSE 0 END) AS approved_total_amount
FROM 
    Transactions
GROUP BY 
    month, country; 

In [ ]:
df = pd.read_csv('data/transactions.csv')

# 2) group by country/month 
# 3) COUNT transactions, SUM amount
# 4) COUNT transactions, SUM amount   IF state = approved

# 1) Create column month
df['month'] = df['trans_date'].dt.strftime("%Y-%m")

# Create 'approved' column with amounts for approved transactions only
df['approved'] = np.where(df['state'] == 'approved', df['amount'], np.nan)

# Group by 'month' and 'country' and perform the aggregation
df.groupby(['month', 'country']).agg(
        trans_count=('state', 'count'),
        approved_count=('approved', 'count'),
        trans_total_amount=('amount', 'sum'),
        approved_total_amount=('approved', 'sum')
    ).reset_index()

In [ ]:
df = pd.read_csv('data/transactions.csv')
sqldf(''' 
	SELECT  
		SUBSTR(trans_date,1,7) as month, 
		country, 
		count(id) as trans_count, 
		SUM(state ='approved') as approved_count, 
		SUM(amount) as trans_total_amount, 
		SUM((state = 'approved') * amount) as approved_total_amount
	FROM df
	GROUP BY month, country
	''')

### Problem: 1934. Confirmation Rate

**Table**: `Signups`

| Column Name | Type     |
|-------------|----------|
| user_id     | int      |
| time_stamp  | datetime |

- `user_id` is the column of **unique values** for this table.
- Each row contains information about the **signup time** for the user with `user_id`.

**Table**: `Confirmations`

| Column Name | Type     |
|-------------|----------|
| user_id     | int      |
| time_stamp  | datetime |
| action      | ENUM     |

- `(user_id, time_stamp)` is the **primary key** (combination of columns with unique values) for this table.
- `user_id` is a **foreign key** (reference column) to the `Signups` table.
- `action` is an **ENUM** (category) of the type `('confirmed', 'timeout')`.
- Each row of this table indicates that the user with `user_id` requested a **confirmation message** at `time_stamp` and that the confirmation message was either **confirmed ('confirmed')** or **expired without confirming ('timeout')**.

### Task:
The **confirmation rate** of a user is the number of `'confirmed'` messages divided by the **total number** of requested confirmation messages. The confirmation rate of a user that did not request any confirmation messages is **0**. **Round the confirmation rate to two decimal places**.

Write a solution to find the **confirmation rate** of each user.

### Example:

**Input**: 
`Signups` table:

| user_id | time_stamp          |
|---------|---------------------|
| 3       | 2020-03-21 10:16:13 |
| 7       | 2020-01-04 13:57:59 |
| 2       | 2020-07-29 23:09:44 |
| 6       | 2020-12-09 10:39:37 |

`Confirmations` table:

| user_id | time_stamp          | action    |
|---------|---------------------|-----------|
| 3       | 2021-01-06 03:30:46 | timeout   |
| 3       | 2021-07-14 14:00:00 | timeout   |
| 7       | 2021-06-12 11:57:29 | confirmed |
| 7       | 2021-06-13 12:58:28 | confirmed |
| 7       | 2021-06-14 13:59:27 | confirmed |
| 2       | 2021-01-22 00:00:00 | confirmed |
| 2       | 2021-02-28 23:59:59 | timeout   |

**Output**:

| user_id | confirmation_rate |
|---------|-------------------|
| 6       | 0.00              |
| 3       | 0.00              |
| 7       | 1.00              |
| 2       | 0.50              |

**Explanation**: 
- **User 6** did not request any confirmation messages. The confirmation rate is **0**.
- **User 3** made **2 requests** and both **timed out**. The confirmation rate is **0**.
- **User 7** made **3 requests** and all were **confirmed**. The confirmation rate is **1**.
- **User 2** made **2 requests** where **one was confirmed** and **the other timed out**. The confirmation rate is **1 / 2 = 0.5**.

In [ ]:
WITH ConfirmationCounts AS (
    SELECT 
        user_id,
        SUM(CASE WHEN action = 'confirmed' THEN 1 ELSE 0 END) AS confirmed_count,
        COUNT(*) AS total_count
    FROM Confirmations
    GROUP BY user_id
)
SELECT 
    s.user_id,
    ROUND(COALESCE(confirmed_count / NULLIF(total_count, 0), 0), 2) AS confirmation_rate
FROM Signups s
LEFT JOIN ConfirmationCounts c ON s.user_id = c.user_id
ORDER BY s.user_id;

In [ ]:
def confirmation_rate(signups: pd.DataFrame, confirmations: pd.DataFrame) -> pd.DataFrame:


    # Create auxiliary columns to count the total number of confirmation requests
    # and the number of successful confirmations (where action is 'confirmed')
    confirmations['total'] = 1
    confirmations['confirmed'] = np.where(confirmations['action'] == 'confirmed',
    1,0)

     # Merge the signups with confirmations to include users who signed up but
     # haven't requested confirmation
    confirmations = pd.merge(confirmations, signups[['user_id']], how = 'outer',
    on = 'user_id')

    # Aggregate the total and successful confirmation counts per user
    result = confirmations.groupby('user_id').agg(
        sum_total   = ('total','sum'),
        sum_confirmed = ('confirmed','sum')
    ).reset_index()

    # Compute confirmation rate
    result['confirmation_rate'] = result['sum_confirmed']/result['sum_total']

    # Get dataframe in the desired format
    result = result[['user_id', 'confirmation_rate']]

    # Round result, as required
    result['confirmation_rate'] =    result['confirmation_rate'].round(2)
    
    # Fill NA values with 0 for users who signed up but did not request confirmation
    result = result.fillna(0)

    return result


# Read mock data
signups = pd.read_csv('data/1934_a_signups.csv')
confirmations = pd.read_csv('data/1934_b_confirmations.csv')

# Call function
confirmation_rate(signups, confirmations)


### Problem: 570. Managers with at Least 5 Direct Reports

**Table**: `Employee`

| Column Name | Type    |
|-------------|---------|
| id          | int     |
| name        | varchar |
| department  | varchar |
| managerId   | int     |

- `id` is the **primary key** (column with unique values) for this table.
- Each row of this table indicates the **name** of an employee, their **department**, and the `id` of their **manager**.
- If `managerId` is **null**, then the employee **does not have a manager**.
- No employee will be the **manager of themself**.

### Task:
Write a solution to find **managers** with **at least five direct reports**.

Return the result table in **any order**.

### Example:

**Input**: 
`Employee` table:

| id  | name  | department | managerId |
|-----|-------|------------|-----------|
| 101 | John  | A          | null      |
| 102 | Dan   | A          | 101       |
| 103 | James | A          | 101       |
| 104 | Amy   | A          | 101       |
| 105 | Anne  | A          | 101       |
| 106 | Ron   | B          | 101       |

**Output**:

| name |
|------|
| John |

**Explanation**: 
- **John** (id `101`) has **5 direct reports** (`Dan`, `James`, `Amy`, `Anne`, `Ron`), so he qualifies as a manager with at least **five direct reports**.

Sketch

1. groupby manager -> count()
2. filter count > 5 
2. recoveer its name. left join with employe table

In [ ]:
SELECT name 
FROM Employee 
WHERE id in (SELECT managerID FROM employee GROUP BY managerId HAVING count(*)>=5);

In [ ]:
employee = pd.read_csv('data/570_employee.csv')


def find_managers(employee: pd.DataFrame) -> pd.DataFrame:
    """
    This function returns a DataFrame containing the names of all managers that
     have at least 5 direct reports. 
    """

    # Count the number of direct reports of each manager
    aux = employee.groupby('managerId').agg(
        direct_reports = ('managerId', 'count')
    ).reset_index()

    # Filter for managers with at least 5 direct reports
    aux = aux[ aux['direct_reports'] >= 5]

    # Get the names of those managers
    result = pd.merge(
        aux,
        employee,
        how = 'left',
        left_on='managerId',
        right_on = 'id'
    )

    return result[['name']]

find_managers(employee)